# Decoding with Belief Propagation - Working to take some for loops out of the game

We want to iterate the following the following equations:

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu } \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$  

which are Eqs.(96) from the paper [Low-density parity-check codes—A statistical physics perspective](https://www.sciencedirect.com/science/article/pii/S1076567002800180), by R. Vicente, D. Saad and Y. Kabashima.

The function $ \beta(x) $ is the Nishimori temperature,

$$ \beta(x) = \frac{1}{2} \log \left(  \frac{1- \rho}{\rho}  \right) \; \; .  $$

The quantity $\rho$ is the flip probablility of the noisy channel (BSC),

$$ P ( J | J^{(0)} ) = (1 - \rho) \delta_{J, J^{(0)} } + \rho \delta_{J, -J^{(0)} }   \; \; , $$

whereas the prior distribution for each meassage bit is assumed to be

$$ P ( S_j ) = (1 - \rho_\xi) \delta_{+1, S_j }  +  \rho_\xi \delta_{-1, S_j }  \; \; .  $$

The object $ {\cal L} (\mu)$ represents the set of $K$ non-zero elements on the row $\mu$ of the code generator matrix ${\cal G}$ (the one which adds redundancy), 

$$  {\cal L} (\mu) = \langle i_1, i_2, ..., i_K \rangle  \; \; .  $$

The are $C$ non-zero elements per column on the matrix ${\cal G}$:

$$ \sum_{\mu : j \in {\cal L}(\mu)} i_j = C \; \; ; \; \; \forall j = 1, ..., K \; \; . $$

The object $ {\cal M} (j)$ represents the set of all index sets that contain $j$.

After convergence of the iterative procedure, we can calculate the pseudo-posterior:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$
from which the Bayes optimal estimate is obtained
$$ \hat{\xi}_j  = sign( m_j ) $$ 

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Defining the variables of the problem:

In [2]:
# Message lengh
N = 10

# Codeword lengh
M = 20

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 15

# Noisy channel
p = 0.3
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.1
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

## Generating messages

Each message is a $N$ dimensional vector. Generate a set of $n$ messages.

In [3]:
random = torch.rand([n, N])

#### -1 with probability p_prior
#### +1 with probability 1 - p_prior

message = 2* (random > p_prior).float() - 1

In [4]:
print(message)
message.shape

tensor([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [-1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.],
        [-1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.]])


torch.Size([15, 10])

## Encoding

Each message is encoded to a high dimensional vector ${\bf J}^{(0)} \in \{ \pm 1  \}^M$ defined as 

$$   J^{(0)}_{\langle i_1, i_2, ...., i_K \rangle} = \xi_1 \xi_ 2 ... \xi_K  \; \; ,$$

where $M$ sets of $K \in [ 1, ..., N]$ indexes are randomly chosen.

In [5]:
encoding = torch.randint(0, N, [M, K])

In [6]:
encoding.shape

torch.Size([20, 4])

From `encoding`, we construct the encoded message ${\bf J}^{(0)}$.

In [7]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

# Loop over all messages
for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

In [8]:
print(J0)
J0.shape

tensor([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,
         -1.,  1., -1., -1., -1., -1.],
        [ 1., -1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1.,
          1.,  1.,  1., -1., -1., -1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [-1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,
          1.,  1., -1., -1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,

torch.Size([15, 20])

## Corrupted version of the encoded set of messages

In [11]:
random = torch.rand(J0.shape)

## Flip tensor: -1 with probability p
flip = 2*(random <= p).float() - 1

## J corrupted version: element wise multiplication
J = torch.mul(J0, flip)

In [12]:
print(J)
J.shape

tensor([[ 1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,
         -1.,  1., -1.,  1., -1.,  1.],
        [-1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,
         -1., -1.,  1.,  1., -1.,  1.],
        [-1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,
          1.,  1.,  1.,  1., -1., -1.],
        [-1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,
         -1., -1., -1., -1., -1.,  1.],
        [ 1.,  1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,
          1., -1.,  1.,  1.,  1.,  1.],
        [ 1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,
          1., -1., -1., -1.,  1., -1.],
        [ 1., -1., -1., -1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,
         -1., -1., -1., -1., -1., -1.],
        [-1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,
          1., -1.,  1.,  1., -1.,  1.],
        [-1., -1., -1., -1., -1., -1., -1., -1.,

torch.Size([15, 20])

## Iterating BP equations for one message

Let us focus in one received message to iterate the belief propagation equations.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)    \right)   \; \; ,  $$  

with $j = 1, ..., N$ and $\mu = 1, ..., M$.

We cal this message `J_`. We will worry later about a loop over all the received messages.

In [13]:
J_ = J[0]
print(J_)
print(J_.shape)

tensor([ 1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,
        -1.,  1., -1.,  1., -1.,  1.])
torch.Size([20])


Random initialization of the beliefs $m_{\mu l}$.

In [14]:
m = torch.rand(M, N)

In [15]:
m

tensor([[0.8614, 0.7178, 0.3371, 0.2367, 0.1067, 0.5971, 0.4043, 0.2510, 0.5894,
         0.4300],
        [0.0165, 0.5987, 0.3994, 0.2361, 0.0408, 0.9143, 0.4325, 0.9045, 0.4464,
         0.6526],
        [0.6426, 0.9224, 0.6518, 0.6517, 0.0502, 0.3809, 0.4694, 0.8129, 0.9538,
         0.7884],
        [0.7773, 0.3005, 0.6068, 0.7846, 0.2382, 0.3705, 0.5191, 0.5133, 0.4931,
         0.2246],
        [0.9193, 0.4419, 0.2408, 0.3896, 0.0336, 0.9566, 0.9059, 0.2318, 0.9567,
         0.0481],
        [0.1630, 0.9127, 0.7098, 0.6659, 0.2184, 0.1310, 0.6776, 0.1232, 0.8820,
         0.4901],
        [0.8475, 0.1713, 0.4835, 0.6974, 0.3424, 0.8972, 0.8632, 0.9342, 0.1468,
         0.3807],
        [0.0481, 0.6787, 0.2059, 0.8278, 0.8066, 0.0966, 0.5245, 0.7966, 0.2402,
         0.1617],
        [0.7684, 0.3427, 0.9070, 0.4991, 0.2772, 0.5889, 0.8014, 0.0127, 0.4912,
         0.6470],
        [0.3333, 0.1253, 0.2880, 0.7476, 0.3486, 0.5392, 0.0883, 0.9343, 0.5379,
         0.3550],
        [0

Initialize an empty tensor to represent $\hat{m}_{\mu l}$.

In [16]:
m_hat = torch.empty(M, N)

In [17]:
m_hat

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  8.6047e-05,
          1.6707e-19,  1.9661e-13,  4.5755e-41,  1.9411e-13,  4.5755e-41],
        [-3.2150e-11,  9.6386e-37,  1.9404e-13,  4.5755e-41,  1.3593e-43,
          0.0000e+00, -3.5438e+00,  3.0719e-41, -3.5352e+00,  3.0719e-41],
        [ 1.4013e-45,  0.0000e+00,  3.4363e-07,  2.2088e-26,  1.9402e-13,
          4.5755e-41,  1.8925e-13,  4.5755e-41,  3.7998e-20,  1.0535e-17],
        [ 4.6243e-44,  0.0000e+00, -3.5355e+00,  3.0719e-41, -3.5354e+00,
          3.0719e-41,  6.5020e-43,  0.0000e+00,  1.5695e-43,  0.0000e+00],
        [-3.5353e+00,  3.0719e-41,  2.7569e-13,  4.5755e-41,  1.9477e-13,
          4.5755e-41,  7.2113e-19, -2.8305e+19,  1.8969e-13,  4.5755e-41],
        [ 1.9478e-13,  4.5755e-41, -6.6884e+24, -1.0948e+27,  4.2997e-13,
          4.5755e-41,  3.9595e-13,  4.5755e-41,  4.8372e-12,  2.2151e+36],
        [ 4.1639e-13,  4.5755e-41,  1.9412e-13,  4.5755e-41,  2.9086e+19,
         -3.9255e+35,  4.8430e-4

We want to calculate $\hat{m}_{\mu j}$.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

This first implementation has two `for` loops. This is potentially harmful if one cares about efficienty. We obviously do, but since we are just beginning, lets go on like this.

In [18]:
for mu in range(M):
    for j in range(N):
          
        # Keep only L(mu) which a are different of j
        index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
        L_no_j = encoding[mu][index_no_j]
        
        # Message update        
        m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)

In [ ]:
m_hat

The next step is to implement:

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)     \right)  \; \; ,  $$ 

In [42]:
for j in range(N):
    
    for mu in range(M):
        
        M_set = torch.where(encoding == j)[0]
        
        M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
        
        m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)

In [43]:
m

tensor([[0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        ...,
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000]])

The next step is to implement the pseudo-posterior, which is calculated after convergence of the messages above:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$

In [44]:
m_bayes = []

for j in range(N):
    
    M_set = torch.where(encoding == j)[0]
       
    m_j = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set).sum() + beta_prior)
    
    m_bayes.append(m_j.item())
            
m_bayes

[0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,


In [45]:
len(m_bayes)

100

In [46]:
np.sign(m_bayes)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Gathering all message iterations on functions:

In [2]:
def m_to_mhat(N, M, J_, m, beta, encoding):
    
    m_hat = torch.empty(M, N)
    
    for mu in range(M):
        for j in range(N):
                     
            # Keep only L(mu) which a are different of j
            index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
            L_no_j = encoding[mu][index_no_j]
        
            # Message update        
            m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)
            
    return m_hat


#####################################################################


def mhat_to_m(N, M, m_hat, beta_prior, encoding):
    
    m = torch.empty(M, N)
    
    for j in range(N):
        
        for mu in range(M):
            
            M_set = torch.where(encoding == j)[0]
        
            M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
            
            m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)
            
    return m


#####################################################################


def m_bayes(N, m_hat_final, beta_prior, encoding):
    
    m_bayes = []

    for j in range(N):
           
        M_set = torch.where(encoding == j)[0]
  
        m_j = torch.tanh(torch.take(np.arctanh(m_hat_final[:, j]), M_set).sum() + beta_prior)
    
        m_bayes.append(m_j.item())
        
    return m_bayes

Now we construct a function for the iterative decoding.

In [3]:
def BP_LDPC(N, M, J_, beta, beta_prior, encoding, message, num_it= 100, verbose= 1):
     
    m_hat = torch.rand(M, N)
    m = torch.empty(M, N)
    
    for j in range(num_it):
        
        print('--it = %d' % j)
              
        m = mhat_to_m(N, M, m_hat, beta_prior, encoding)
        m_hat = m_to_mhat(N, M, J_, m, beta, encoding)
        
        ## Monitoring performace
        if (j % verbose) == 0:
            m_ = m_bayes(N, m_hat, beta_prior, encoding)
            print('overlap= ', torch.dot(message, torch.Tensor(np.sign(m_))).item() / N)
        
    m_final = m_bayes(N, m_hat, beta_prior, encoding)
    
    return np.sign(m_final)

In [49]:
opt_dec_Bayes = BP_LDPC(N, M, J_, beta, beta_prior, encoding, message[0], num_it= 20, verbose= 1)

--it = 0
overlap=  0.76
--it = 1
overlap=  0.76
--it = 2
overlap=  0.76
--it = 3
overlap=  0.76
--it = 4
overlap=  0.76
--it = 5
overlap=  0.76
--it = 6
overlap=  0.76
--it = 7
overlap=  0.76
--it = 8
overlap=  0.76
--it = 9
overlap=  0.76
--it = 10
overlap=  0.76
--it = 11
overlap=  0.76
--it = 12
overlap=  0.76
--it = 13
overlap=  0.76
--it = 14
overlap=  0.76
--it = 15
overlap=  0.76
--it = 16
overlap=  0.76
--it = 17
overlap=  0.76
--it = 18
overlap=  0.76
--it = 19
overlap=  0.76


Ok. It is still slow (we will worrie about this latter), but it appears to work.

Observe that we arbitrarily consider a certain `num_it`. Naturally, a important question remains: how do we monitor the convergence of BP?

## Quick test on previous codes and messages

In [4]:
# Message lengh
N = 250

# Codeword lengh
M = 500

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 100

# Noisy channel
p = 0.3
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.01
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

In [5]:
encoding = torch.load('codes/code_N_250_M_500_K_4_p_03_p_prior_001_j_0.pt')
message = torch.load('codes/message_N_250_M_500_K_4_p_03_p_prior_001.pt')

In [6]:
encoding

tensor([[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])

In [7]:
print(message)
message.shape

tensor([[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])


torch.Size([100, 250])

Well, we made a mistake saving the codes. We have actually saved the messages. That is a problem. For now, let us generate a new code.

In [8]:
encoding  = torch.randint(0, N, [M, K])
encoding

tensor([[ 19,  14, 131, 126],
        [  0,   0, 169, 179],
        [226, 164, 236, 118],
        ...,
        [ 47, 104, 136, 229],
        [173, 132,   4, 112],
        [ 41, 106,  31, 195]])

Encoding.

In [9]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

In [10]:
print(J0)
J0.shape

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


torch.Size([100, 500])

Corrupted version.

In [11]:
random = torch.rand(J0.shape)

## Flip tensor: -1 with probability p
flip = 2*(random <= p).float() - 1

## J corrupted version: element wise multiplication
J = torch.mul(J0, flip)

In [12]:
print(J)
J.shape

tensor([[ 1.,  1., -1.,  ..., -1., -1.,  1.],
        [ 1.,  1., -1.,  ..., -1., -1., -1.],
        [ 1.,  1., -1.,  ..., -1., -1.,  1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1.,  1.],
        [-1., -1.,  1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])


torch.Size([100, 500])

In [13]:
J[0].shape

torch.Size([500])

In [14]:
message[0].shape

torch.Size([250])

In [15]:
print(N)
print(M)
print(encoding.shape)
print(J.shape)
print(message.shape)

250
500
torch.Size([500, 4])
torch.Size([100, 500])
torch.Size([100, 250])


In [16]:
overlap_list = []

for k in range(int(J.shape[0] / 10 )):
    
    print('----message %d' % k)
       
    opt_dec_Bayes = BP_LDPC(N, M, J[k], beta, beta_prior, encoding, message[k], num_it= 10, verbose= 1)
    
    overlap_list.append(opt_dec_Bayes)

----message 0
--it = 0
overlap=  0.648
--it = 1
overlap=  0.992
--it = 2
overlap=  0.808
--it = 3
overlap=  0.992
--it = 4
overlap=  0.832
--it = 5
overlap=  0.992
--it = 6
overlap=  0.848
--it = 7
overlap=  0.992
--it = 8
overlap=  0.864
--it = 9
overlap=  0.992
----message 1
--it = 0
overlap=  0.656
--it = 1
overlap=  0.976
--it = 2


KeyboardInterrupt: 

## Loop over different noise parameters to chech consistency

In [20]:
# Message lengh
N = 250

# Codeword lengh
M = 500

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 100

# Noisy channel
p = np.array([0.1, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.45, 0.495])
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.01
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

Generating a message.

In [23]:
random = torch.rand([n, N])

#### -1 with probability p_prior
#### +1 with probability 1 - p_prior

message = 2* (random > p_prior).float() - 1

Encoding.

In [25]:
encoding = torch.randint(0, N, [M, K])

In [26]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

# Loop over all messages
for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

Loop.

In [ ]:
random = torch.rand(J0.shape)

p_list = []
overlap_list = []

for p_ in p:
    
    print('------ p = ', p_)
    
    p_list.append(p_)
    
    beta_ = 0.5*np.log( (1 - p_) / p_)
    
    ## Flip tensor: -1 with probability p
    flip = 2*(random <= p_).float() - 1
    ## J corrupted version: element wise multiplication
    J = torch.mul(J0, flip)
    ##
    overlap_list_p = []

    for k in range(int(J.shape[0] / 10 )):
    
        print('----message %d' % k)
       
        opt_dec_Bayes = BP_LDPC(N, M, J[k], beta_, beta_prior, encoding, message[k], num_it= 10, verbose= 1)
    
        overlap_list_p.append(opt_dec_Bayes)
        
    overlap_list.append(overlap_list_p)

------ p =  0.1
----message 0
--it = 0
